In [1]:
#!wget -r -A.html -P rtdocs https://langchain.readthedocs.io/en/latest/

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [18]:
from langchain.document_loaders import ReadTheDocsLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

from virtual_assistant.utilities import GoogleSerperAPIWrapper, initialize_pinecone

In [15]:
initialize_pinecone()

True

In [16]:
loader = ReadTheDocsLoader("rtdocs")
raw_documents = loader.load()
raw_documents[3]

Document(page_content='.md\n.pdf\nTracing\n Contents \nTracing Walkthrough\nChanging Sessions\nTracing#\nBy enabling tracing in your LangChain runs, you’ll be able to more effectively visualize, step through, and debug your chains and agents.\nFirst, you should install tracing and set up your environment properly.\nYou can use either a locally hosted version of this (uses Docker) or a cloud hosted version (in closed alpha).\nIf you’re interested in using the hosted platform, please fill out the form here.\nLocally Hosted Setup\nCloud Hosted Setup\nTracing Walkthrough#\nWhen you first access the UI, you should see a page with your tracing sessions.\nAn initial one “default” should already be created for you.\nA session is just a way to group traces together.\nIf you click on a session, it will take you to a page with no recorded traces that says “No Runs.”\nYou can create a new session with the new session form.\nIf we click on the default session, we can see that to start we have no tr

In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(raw_documents)
embeddings = OpenAIEmbeddings()
vectorstore = Pinecone.from_documents(documents, embeddings, index_name="personal-knowledgebase")

In [30]:
vectorstore = Pinecone.from_existing_index(index_name='personal-knowledgebase', embedding=embeddings)

In [35]:
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [36]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [37]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), return_source_documents=True)

In [43]:
chat_history = []

In [44]:
query = "What's langchain latest version code"
result = qa({"question": query, "chat_history": chat_history})